# Write and Document Benchmark Examples for NeuralNetDiffEq.jl

In [ ]:
using OrdinaryDiffEq, NeuralNetDiffEq, Plots, Flux

### Example: Lotka-Voltera 
The Lotka-Volterra equations, also the predator-prey equations, models changes in population of prey and predator over time when they interact. This is an example of a first-order nonlinear coupled differential equations, described as:

$$\frac{dx}{dt}=\alpha x -\beta x y, \; \frac{dy}{dt}=\delta x y-\gamma y$$
where 
$x$ and $y$ are the prey and predator populations; $\frac{dx}{dt}$ and $\frac{dy}{dt}$ denote instantaneous growth rates. This interaction is parameterised by positive real parameters $\alpha, \beta, \gamma, \delta$

In [ ]:
function f(du,u,p,t)
  du[1] = p[1]*u[1] - p[2]*u[1]*u[2]
  du[2] = -p[3]*u[2] + p[4]*u[1]*u[2]
end
function f(u,p,t)
  [p[1]*u[1] - p[2]*u[1]*u[2],-p[3]*u[2] + p[4]*u[1]*u[2]]
end

In [ ]:
p = Float32[1.5,1.0,3.0,1.0]
u0 = Float32[1.0,1.0]
prob = ODEProblem(f,u0,(0f0,3f0),p)
prob_oop = ODEProblem{false}(f,u0,(0f0,3f0),p)

### Defining Model and Optimiser
After many experiments, Adam and NADAM optimiser (with eta=1e-03) tend to outperform others, achieveing a loss as low as 44.6 in 100 epochs. All methods plateau after this loss, if it is reached at all. Larger models with up to 4096 channels tend to plateau at 51 loss within 100 iterations, while smaller models fail to achieve this accuracy. MaxPool layers do not seem to add value to the model.

In [ ]:
true_sol = solve(prob,Tsit5())

opt = ADAM(1e-03) #1e-04
# opt = NADAM()
# opt = Nesterov()
# opt = AMSGrad()
# chain = Chain(x -> reshape(x, length(x), 1, 1), Conv((1,), 1=>16, relu), Conv((1,), 16=>8, relu), x -> reshape(x, :, size(x, 4)), Dense(8, 10), softmax)


chain = Chain(
    x -> reshape(x, length(x), 1, 1), 
    MaxPool((1,)), 
    Conv((1,), 1=>16, relu), 
    Conv((1,), 16=>16, relu), 
    Conv((1,), 16=>32, relu), 
    Conv((1,), 32=>64, relu), 
    Conv((1,), 64=>256, relu), 
    Conv((1,), 256=>256, relu), 
    Conv((1,), 256=>1028, relu), 
    Conv((1,), 1028=>1028), 
    x -> reshape(x, :, size(x, 4)), 
    Dense(1028, 512, tanh), 
    Dense(512, 128, relu), 
    Dense(128, 64, tanh), 
    Dense(64, 2), 
    softmax)

# m = Chain(Conv((1,), 1=>16, relu), Conv((1,), 16=>8, relu), x -> reshape(x, :, size(x, 4)), Dense(16, length(u0)), softmax) 

In [ ]:
sol  = solve(prob_oop,NeuralNetDiffEq.NNODE(chain,opt),maxiters = 100, verbose = true, dt=1/5f0)

In [3]:
1e-03

0.001

### Plotting

In [ ]:
plot(true_sol)
plot!(sol)
# savefig("ADAM.png")